In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [9]:
df = pd.read_csv('Preprocessed_data.csv')


df.index = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df.drop(['Date', 'Salmon_exp_price'], axis=1, inplace=True)

In [14]:
df

,Salmon_exp_vol,FX_EURONOK,Interest_rate,Food_index
Date,,,,
2001-01-01,4089,8.27300,5.50,59.936571
2001-01-08,4959,8.20900,5.50,59.936571
2001-01-15,4610,8.27250,5.50,59.936571
2001-01-22,4470,8.20700,5.50,59.936571
2001-01-29,4523,8.19600,5.50,59.936571
...,...,...,...,...
2023-11-27,22910,11.61061,2.25,114.626222
2023-12-04,22243,11.73763,2.50,112.403991
2023-12-11,24551,11.42400,2.50,112.403991


In [27]:
y = df['Salmon_exp_vol']
X = df.loc[:,'FX_EURONOK':]

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=0.2,
                                                    shuffle=False)

In [44]:
# Manual change in FX column
X_train['FX_EURONOK']['2002-10-14'] = 7.34


# Log transform  the FX columns
def log_transform(X_train, X_test):
    X_train = np.log(X_train)
    X_test = np.log(X_test)
    return X_train, X_test